### Infrastructure as Code (IaC) with Boto3

1. Define Infrastructure with Boto3 (Python Script):

   - Use Boto3 to define the infrastructure (VPC, subnets, security groups).

   - Define an Auto Scaling Group (ASG) for the backend.

   - Create AWS Lambda functions if needed.

In [ ]:
pip install boto3

In [ ]:
import boto3

created_vpc_id = None
created_security_group_id = None

def create_infrastructure():
    ec2 = boto3.resource('ec2', region_name='ap-northeast-2')


    vpc = ec2.create_vpc(CidrBlock='10.0.0.0/27')
    vpc.create_tags(Tags=[{"Key": "Name", "Value": "micromernaws_vpc"}])
    vpc.wait_until_available()

    print(f"VPC ID: {vpc.id}")
    print(f"CIDR Block: {vpc.cidr_block}")
    

    security_group = ec2.create_security_group(
        GroupName='micromernaws_sg',
        Description='Security group for micromernaws',
        VpcId=vpc.id,
        TagSpecifications=[
            {
                'ResourceType': 'security-group',
                'Tags': [
                    {'Key': 'Name', 'Value': 'micromernaws_sg'}
                ]
            }
        ]
    )

    def attach_subnets(vpc):
        cidr_blocks = ['10.0.0.0/28', '10.0.0.16/28'] 
        availability_zones = ['ap-northeast-2a', 'ap-northeast-2b']
        subnets = []

        for i, az in enumerate(availability_zones):
            subnet = ec2.create_subnet(
                CidrBlock=cidr_blocks[i],  
                VpcId=vpc.id,
                AvailabilityZone=az
            )
            subnets.append(subnet)
            print(f"Subnet created in {az}: {subnet.id}")
        
        return subnets

    attach_subnets(vpc)
    
    ec2.meta.client.authorize_security_group_ingress(
        GroupId=security_group.id,
        IpPermissions=[
            {
                'IpProtocol': 'tcp',
                'FromPort': 80,
                'ToPort': 80,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]  
            },
            {
                'IpProtocol': 'tcp',
                'FromPort': 443,
                'ToPort': 443,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]  
            },
            {
                'IpProtocol': 'tcp',
                'FromPort': 22,
                'ToPort': 22,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}] 
            },
            {
                'IpProtocol': 'tcp',
                'FromPort': 3001,
                'ToPort': 3002,
                'IpRanges': [{'CidrIp': '0.0.0.0/0'}]  
            }
        ]
    )
    
    print(f"Security Group ID: {security_group.id}")
    return vpc.id, security_group.id

created_vpc_id, created_security_group_id = create_infrastructure()


### create template image from instance

1. Required Instance image_id

In [ ]:
import boto3

imageclient = boto3.client('ec2', region_name='ap-northeast-2')

imagecreateedres = imageclient.create_image(
    InstanceId='i-0d5a93331d372682f',
    Name='micromernaws_image',
    TagSpecifications=[
        {
            'ResourceType': 'image',
            'Tags': [
                {'Key': 'Name', 'Value': 'micromernaws_image'}
            ]
        }
    ]
)

print(f"Image Created: {imagecreateedres['ImageId']}")


## WAIT PERIOD OF 5-10 MIN DEPENDING ON INSTANCE

In order to create ASG we need Instance Image or template 
- We can either create it manually by having existing instance
- Else we can automate build by creating instance with boto3 including userData with executable steps

In [ ]:
import boto3

ec2_client = boto3.client('ec2', region_name='ap-northeast-2')

template_res = ec2_client.create_launch_template(
    LaunchTemplateName='micromernbackend_aws',
    VersionDescription='Launch template for micromernbackend',
    LaunchTemplateData={
        'ImageId': 'ami-0b5f26a9a1e3ae54b',  
        'InstanceType': 't2.micro',  
        'SecurityGroupIds': [created_security_group_id],  
        'KeyName': 'ChrisEvans',
        'TagSpecifications': [
            {
                'ResourceType': 'instance',
                'Tags': [
                    {'Key': 'Name', 'Value': 'micromern-backend-instance'}
                ]
            }
        ]
    }
)

# Print the response
print(f"Launch Template Created: {template_res['LaunchTemplate']['LaunchTemplateId']}")


### Autoscaling from launch template

In [ ]:
import boto3

ec2_client = boto3.client('ec2', region_name='ap-northeast-2')
autoscaling = boto3.client('autoscaling', region_name='ap-northeast-2')

subnet_response = ec2_client.describe_subnets(Filters=[{'Name': 'vpc-id', 'Values': [created_vpc_id]}])
subnet_ids = [subnet['SubnetId'] for subnet in subnet_response['Subnets']]

print(f"Retrieved Subnet IDs: {subnet_ids}")

if not subnet_ids:
    raise ValueError(f"No subnets found for VPC {created_vpc_id}. Please ensure the VPC has subnets.")

auto_res = autoscaling.create_auto_scaling_group(
    AutoScalingGroupName='micromernaws',
    LaunchTemplate={
        'LaunchTemplateName': 'micromernbackend_aws',
        'Version': '1'
    },
    MinSize=1,
    MaxSize=3,
    DesiredCapacity=1,
    VPCZoneIdentifier=','.join(subnet_ids),  
    Tags=[
        {
            'Key': 'Name',
            'Value': 'micromernaws_asg',
            'PropagateAtLaunch': True 
        },
    ]
)

print(f"Auto Scaling Group Created: {auto_res}")

elb_client = boto3.client('elbv2', region_name='ap-northeast-2')

load_balancer_response = elb_client.create_load_balancer(
    Name='micromernaws-elb',
    Subnets=subnet_ids,
    SecurityGroups=[created_security_group_id],
    Scheme='internet-facing',
    Tags=[
        {
            'Key': 'Name',
            'Value': 'micromernaws-elb'
        }
    ],
    Type='application',
    IpAddressType='ipv4'
)

load_balancer_arn = load_balancer_response['LoadBalancers'][0]['LoadBalancerArn']
print(f"Load Balancer ARN: {load_balancer_arn}")

target_group_response = elb_client.create_target_group(
    Name='micromernaws-targets',
    Protocol='HTTP',
    Port=80,
    VpcId=created_vpc_id,
    HealthCheckProtocol='HTTP',
    HealthCheckPort='80',
    HealthCheckPath='/',
    HealthCheckIntervalSeconds=30,
    HealthCheckTimeoutSeconds=5,
    HealthyThresholdCount=5,
    UnhealthyThresholdCount=2,
    TargetType='instance'
)

target_group_arn = target_group_response['TargetGroups'][0]['TargetGroupArn']
print(f"Target Group ARN: {target_group_arn}")

listener_response = elb_client.create_listener(
    LoadBalancerArn=load_balancer_arn,
    Protocol='HTTP',
    Port=80,
    DefaultActions=[
        {
            'Type': 'forward',
            'TargetGroupArn': target_group_arn
        }
    ]
)

listener_arn = listener_response['Listeners'][0]['ListenerArn']
print(f"Listener ARN: {listener_arn}")

autoscaling.attach_load_balancer_target_groups(
    AutoScalingGroupName='micromernaws',
    TargetGroupARNs=[target_group_arn]
)

print("ELB setup completed and attached to the Auto Scaling Group.")
load_balancer_dns_name = load_balancer_response['LoadBalancers'][0]['DNSName']
print(f"Load Balancer DNS Name: {load_balancer_dns_name}")

### Completion
* Copy DNS to any Domain service and add it to record and host it